## ETL on `OpenPowerlifting` Data via `duckdb`

In [1]:
from IPython.display import display, Markdown
import polars as pl
from datetime import datetime as dt

# read configs
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from steps import conf

s3_file_path = f"https://{conf.bucket_name}.s3.ap-southeast-2.amazonaws.com/{conf.parquet_file}"

In [24]:
df = pl.read_parquet(s3_file_path)

In [25]:
import re


# functions
def camel_to_snake(camel_str):
    snake_str = re.sub(r"(?<!^)(?=[A-Z])", "_", camel_str).lower()
    return snake_str

In [32]:
df.rename(mapping=remapping_cols)

name,sex,event,equipment,age,age_class,birth_year_class,division,bodyweight_kg,weight_class_kg,squat1_kg,squat2_kg,squat3_kg,squat4_kg,best3_squat_kg,bench1_kg,bench2_kg,bench3_kg,bench4_kg,best3_bench_kg,deadlift1_kg,deadlift2_kg,deadlift3_kg,deadlift4_kg,best3_deadlift_kg,total_kg,place,dots,wilks,glossbrenner,goodlift,tested,country,state,federation,parent_federation,date,meet_country,meet_state,meet_town,meet_name
str,str,str,str,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str
"""Alona Vladi""","""F""","""SBD""","""Raw""",33.0,"""24-34""","""24-39""","""O""",58.3,"""60""",75.0,80.0,-90.0,null,80.0,50.0,55.0,60.0,null,60.0,95.0,105.0,107.5,null,107.5,247.5,"""1""",279.44,282.18,249.42,57.1,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Galina Solovya…","""F""","""SBD""","""Raw""",43.0,"""40-44""","""40-49""","""M1""",73.1,"""75""",95.0,100.0,105.0,null,105.0,62.5,67.5,-72.5,null,67.5,100.0,110.0,-120.0,null,110.0,282.5,"""1""",278.95,272.99,240.35,56.76,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Daniil Voronin…","""M""","""SBD""","""Raw""",15.5,"""16-17""","""14-18""","""T""",67.4,"""75""",85.0,90.0,100.0,null,100.0,55.0,62.5,-65.0,null,62.5,90.0,100.0,105.0,null,105.0,267.5,"""1""",206.4,206.49,200.45,41.24,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Aleksey Krasov…","""M""","""SBD""","""Raw""",35.0,"""35-39""","""24-39""","""O""",66.65,"""75""",125.0,132.0,137.5,null,137.5,115.0,122.5,-127.5,null,122.5,150.0,165.0,170.0,null,170.0,430.0,"""1""",334.49,334.94,325.32,66.68,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Margarita Ples…","""M""","""SBD""","""Raw""",26.5,"""24-34""","""24-39""","""O""",72.45,"""75""",80.0,85.0,90.0,null,90.0,40.0,50.0,-60.0,null,50.0,112.5,120.0,125.0,null,125.0,265.0,"""1""",194.46,193.55,187.29,39.34,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Sergey Timoshe…","""M""","""SBD""","""Raw""",15.5,"""16-17""","""14-18""","""T""",78.8,"""85""",90.0,-95.0,100.0,null,100.0,60.0,-65.0,-67.5,null,60.0,90.0,105.0,115.0,null,115.0,275.0,"""1""",191.34,189.56,182.77,39.09,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Vladimir Karav…","""M""","""SBD""","""Raw""",57.5,"""55-59""","""50-59""","""M2""",79.65,"""85""",50.0,150.0,180.0,null,180.0,50.0,100.0,-110.0,null,100.0,55.0,null,null,null,55.0,335.0,"""1""",231.6,229.34,221.02,47.35,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Tatyana Altuni…","""F""","""SBD""","""Raw""",26.0,"""24-34""","""24-39""","""O""",96.5,"""105""",85.0,95.0,100.0,null,100.0,40.0,47.5,-50.0,null,47.5,115.0,130.0,140.0,null,140.0,287.5,"""1""",248.84,242.04,208.84,51.79,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"
"""Ruslan Gasanov…","""M""","""SBD""","""Raw""",31.5,"""24-34""","""24-39""","""O""",102.55,"""105""",210.0,225.0,232.5,null,232.5,150.0,160.0,-165.0,null,160.0,240.0,260.0,-270.0,null,260.0,652.5,"""1""",397.35,393.25,375.51,81.45,null,"""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournamen…"


In [3]:
import duckdb
import pandas as pd

# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells

In [4]:
%load_ext sql
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False
%sql duckdb:// --alias duckdb-sqlalchemy
%sql duckdb:///:default:

Deploy AI and data apps for free on Ploomber Cloud! Learn more: https://docs.cloud.ploomber.io/en/latest/quickstart/signup.html


Tip: You may define configurations in /Users/ntonthat/github/powerlifting-ml-progress/pyproject.toml or /Users/ntonthat/.jupysql/config.

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Connecting to 'duckdb-sqlalchemy'

Connecting and switching to connection 'duckdb:///:default:'

In [6]:
%%sql
SELECT
    schema_name,
    function_name
FROM duckdb_functions()
ORDER BY ALL DESC
LIMIT 5

,schema_name,function_name
0,pg_catalog,shobj_description
1,pg_catalog,shobj_description
2,pg_catalog,shobj_description
3,pg_catalog,pg_typeof
4,pg_catalog,pg_typeof


In [9]:
sql_query = open("../sqls/01_read_and_create_landing_table.sql").read()

In [15]:
read_and_create_landing_table_query = sql_query.format(
    s3_file_path=s3_file_path,
)
print(read_and_create_landing_table_query)

CREATE SCHEMA IF NOT EXISTS landing;
CREATE TABLE landing.openpowerlifting AS (
    SELECT
        *
    FROM read_parquet('https://powerlifting-ml-progress.s3.ap-southeast-2.amazonaws.com/openpowerlifting-latest.parquet')
);


In [15]:
read_parquet = f"SELECT * FROM read_parquet('{s3_file_path}');"

read_and_create_landing_table = f"""
CREATE SCHEMA IF NOT EXISTS landing;
CREATE TABLE landing.openpowerlifting AS (
    SELECT
        *
    FROM read_parquet('{s3_file_path}')
);
"""

In [16]:
%%sql
CREATE SCHEMA IF NOT EXISTS landing;
CREATE TABLE landing.openpowerlifting AS (
    SELECT
        *
    FROM read_parquet('https://powerlifting-ml-progress.s3.ap-southeast-2.amazonaws.com/openpowerlifting-latest.parquet')
);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [17]:
%sql select * from landing.openpowerlifting limit 10;

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Alona Vladi,F,SBD,Raw,33.0,24-34,24-39,O,58.30,60,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
1,Galina Solovyanova,F,SBD,Raw,43.0,40-44,40-49,M1,73.10,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
2,Daniil Voronin,M,SBD,Raw,15.5,16-17,14-18,T,67.40,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
3,Aleksey Krasov,M,SBD,Raw,35.0,35-39,24-39,O,66.65,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
4,Margarita Pleschenkova,M,SBD,Raw,26.5,24-34,24-39,O,72.45,75,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
5,Sergey Timoshenko,M,SBD,Raw,15.5,16-17,14-18,T,78.80,85,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
6,Vladimir Karavaev,M,SBD,Raw,57.5,55-59,50-59,M2,79.65,85,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
7,Tatyana Altunina,F,SBD,Raw,26.0,24-34,24-39,O,96.50,105,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
8,Ruslan Gasanov,M,SBD,Raw,31.5,24-34,24-39,O,102.55,105,...,None,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
9,Ekaterina Glukhova,F,SBD,Wraps,25.5,24-34,24-39,O,54.80,55,...,Yes,Russia,None,GFP,None,2019-05-11,Russia,None,Bryansk,Open Tournament
